# Distribution Grid Modeling Using AltDSS

Make sure to copy two python files from https://github.com/dss-extensions/AltDSS-Schema/tree/main/python and paste it inside `01-altdss` folder.

We are going to model following distribution model.

![image.png](sld.png)

In [1]:
from altdss_models import Vsource_MVAsc3MVAsc1x1r1x0r0, Bus, BusConnection

bus1 = Bus(Name="Bus1", kVLL=13.2)
bus2 = Bus(Name="Bus2", kVLL=13.2)
bus3 = Bus(Name="Bus3", kVLL=13.2)
vsource = Vsource_MVAsc3MVAsc1x1r1x0r0(
    Name="source",
    Bus1=BusConnection(root="Bus1"),
    BasekV=13.2,
    Phases=3,
    MVASC3=20000.0,
    MVASC1=21000.0,
    pu=1.0001,
    Angle=30.0,
)

In [2]:
from altdss_models import Line_ZMatrixCMatrix, Line_Z0Z1C0C1, SymmetricMatrix

line_1 = Line_ZMatrixCMatrix(
    Name="line_1",
    Bus1=BusConnection(root="Bus1"),
    Bus2=BusConnection(root="Bus2"),
    Phases=3,
    RMatrix=SymmetricMatrix(
        root=[[0.4, 0.05, 0.05], [0.05, 0.4, 0.05], [0.05, 0.05, 0.4]]
    ),
    XMatrix=SymmetricMatrix(root=[[1.2, 0.3, 0.3], [0.3, 1.2, 0.3], [0.3, 0.3, 1.2]]),
    CMatrix=SymmetricMatrix(
        root=[[10.0, -2.0, -2.0], [-2.0, 10.0, -2.0], [-2.0, -2.0, 10.0]]
    ),  # nanofarads (nF/km)
)
line_2 = Line_Z0Z1C0C1(
    Name="line_2",
    Bus1=BusConnection(root="Bus1"),
    Bus2=BusConnection(root="Bus2"),
    Phases=3,
    R1=0.4,  # Ohms/km
    X1=1.2,  # Ohms/km
    R0=0.8,  # Ohms/km
    X0=2.5,  # Ohms/km
    C0=3.0,  # nF/km (typical for overhead lines)
    C1=9.9,  # nF/km
)

In [3]:
from altdss_models import Load_kWkvar, Load_kWPF

load_1 = Load_kWkvar(
    Name="load_1",
    Bus1=BusConnection(root="Bus3"),
    kW=20.0,
    kvar=10.0,
    kV=13.2,
    Phases=3,
)
load_2 = Load_kWPF(
    Name="load_2", Bus1=BusConnection(root="Bus2"), kW=40.0, PF=0.9, kV=13.2, Phases=3
)

In [4]:
from altdss_models import (
    Circuit,
    VsourceContainer,
    LineContainer,
    LoadContainer,
)

circuit = Circuit(
    Name="circuit",
    Bus=[bus1, bus2, bus3],
    Vsource=VsourceContainer.model_validate([vsource.model_dump()]),
    Line=LineContainer.model_validate(list(map(lambda x: x.model_dump(), [line_1, line_2]))),
    Load=LoadContainer.model_validate(list(map(lambda x: x.model_dump(), [load_1, load_2]))),
)

In [5]:
with open("test.dss", "w") as f:
    circuit.dump_dss(f)

In [6]:
circuit.PostCommands = [
    "calcvoltagebases",
    "solve"
]

In [11]:
import opendssdirect as odd
odd.Command("Redirect test.dss")